# Hidden Markov Model para glosado Otomi

In [1]:
from nltk.tag import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pickle import load
from itertools import chain
import pandas as pd
import numpy as np

In [2]:
file1 = load(open('pickle_objects/preinput_data','rb'))
print(file1[0], len(file1))

[[["'", 'v', 'B-como'], ['á', 'v', 'I-como'], ['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['m', 'v', 'B-ctrf'], ['á', 'v', 'I-ctrf'], ['k', 'v', 'B-stem'], ['h', 'v', 'I-stem'], ['á', 'v', 'I-stem']]] 1769


In [3]:
data = []

for s in file1:
    sent_data = []
    for w in s:
        #print(w)
        word = [(x[0], x[2]) for x in w] + [(' ', ' ')]
        sent_data.append(word)
    
    one_sent = list(chain(*sent_data))
    one_sent.pop(-1)
    data.append(list(zip(*one_sent)))

In [4]:
#Data split 33%
train_pairs, eval_pairs = train_test_split(data, test_size=0.33)
print(len(train_pairs), len(eval_pairs))

#Separar el train en dos conjuntos
train_set = [list(zip(s,t)) for s,t in train_pairs]

1185 584


### Entrenamiento del modelo

In [5]:
%%time
trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_set)

CPU times: user 45.2 ms, sys: 777 µs, total: 45.9 ms
Wall time: 46 ms


### Evaluación

In [6]:
y_pred = []
y_true = []
for sent in eval_pairs:
    prediccion = tagger.tag(sent[0])
    y_pred.append(prediccion)
    y_true.append(sent[1])

In [7]:
Y_true = list(chain(*y_true))
Y_pred = list(chain(*y_pred))
chars,Y_pred = zip(*Y_pred) 
Y_pred = list(Y_pred)

In [8]:
labels = list(set(Y_true))
labels.remove(' ')

print('Accuracy:', accuracy_score(Y_true, Y_pred))

Accuracy: 0.8719470722809621


In [9]:
prec, rec, f1, supp = precision_recall_fscore_support(Y_true, Y_pred, labels=labels)

/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
results = pd.DataFrame(data=np.array([prec, rec, f1, supp]).T, index=labels, columns=['Precision','Recall','F1', 'Support'])
results

,Precision,Recall,F1,Support
B-3.icp,0.881356,0.971963,0.924444,107.0
I-dual,0.800000,0.666667,0.727273,6.0
I-dios,0.000000,0.000000,0.000000,2.0
B-uno,0.000000,0.000000,0.000000,1.0
B-det.pl,0.936709,0.961039,0.948718,77.0
...,...,...,...,...
I-1.irr,0.000000,0.000000,0.000000,4.0
I-det.pl,0.913580,0.925000,0.919255,80.0
I-dem,0.232198,0.757576,0.355450,99.0
I-neg,0.500000,0.400000,0.444444,10.0


In [11]:
print(results.to_string())

                  Precision    Recall        F1  Support
B-3.icp            0.881356  0.971963  0.924444    107.0
I-dual             0.800000  0.666667  0.727273      6.0
I-dios             0.000000  0.000000  0.000000      2.0
B-uno              0.000000  0.000000  0.000000      1.0
B-det.pl           0.936709  0.961039  0.948718     77.0
I-prf              0.000000  0.000000  0.000000      1.0
B-ctrf             0.700935  0.949367  0.806452     79.0
B-chente           0.000000  0.000000  0.000000      1.0
B-p.loc            0.666667  0.250000  0.363636      8.0
I-2.cnt            1.000000  0.750000  0.857143      8.0
B-tal.vez          0.000000  0.000000  0.000000      1.0
B-int              0.000000  0.000000  0.000000      2.0
I-hasta            0.000000  0.000000  0.000000      2.0
B-toluca           1.000000  0.666667  0.800000      3.0
B-con              0.000000  0.000000  0.000000      1.0
I-p.loc            0.666667  0.250000  0.363636      8.0
I-1.pss            0.789474  0.

In [12]:
aver_prec = (supp*prec).sum()/supp.sum()
aver_rec = (supp*rec).sum()/supp.sum()
aver_f1 = (supp*f1).sum()/supp.sum()

print('Average precision: {}\nAverage recall: {}\nAverage F1: {}'.format(aver_prec, aver_rec, aver_f1))

Average precision: 0.877801438957231
Average recall: 0.8515518600817146
Average F1: 0.8588883521649597
